In [1]:
import numpy as np # linear algebra
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
from sklearn import linear_model,svm, metrics
import winsound
%matplotlib inline

Load Data from pickle files

In [2]:
folder = 'D:/Libraries/Documents/Tensorflow/tensorflow/Statoil/'
with open(folder+'labeled_data.pkl','rb') as f:
    data = pickle.load(f)

b1train= data['b1train']
b1valid= data['b1valid']
b1test= data['b1test']
b2train= data['b2train']
b2valid= data['b2valid']
b2test= data['b2test']
anglestrain= data['anglestrain']
anglesvalid= data['anglesvalid']
anglestest= data['anglestest']
labelstrain= data['labelstrain']
labelsvalid= data['labelsvalid']
labelstest= data['labelstest']
idstrain= data['idstrain']
idsvalid= data['idsvalid']
idstest= data['idstest']

## Let's try some 'off the shelf' sklearn classifiers. Logistic Regression, SVC

In [3]:
X = np.concatenate((b1train, b2train), axis=1)
Xvalid = np.concatenate((b1valid,b2valid),axis=1)
Xtest = np.concatenate((b1test,b2test),axis=1)

In [4]:
LR = linear_model.LogisticRegression()
LR.fit(X=X,y=labelstrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [5]:
print("Raw Score: " + str(LR.score(Xvalid,labelsvalid)))
pred_proba = LR.predict_proba(Xvalid)
print("Log Loss: " + str(metrics.log_loss(labelsvalid,pred_proba)))

Raw Score: 0.74
Log Loss: 0.763588456889


Logistic Regression Gives:
Raw Score: 0.74
Log Loss: 0.763588456889

In [6]:
kernels = ['rbf', 'linear', 'sigmoid']
for k in kernels:
    SVC = svm.SVC(kernel=k)
    SVC.fit(X=X,y=labelstrain)
    print("kernel = " + str(k))
    print("Raw Score: " + str(SVC.score(Xvalid,labelsvalid)))

kernel = rbf
Raw Score: 0.52


KeyboardInterrupt: 

kernel = rbf
Raw Score: 0.52
kernel = linear
Raw Score: 0.73
kernel = sigmoid
Raw Score: 0.52
    
Seems like the data is fairly linearly separable

In [61]:
Xnorm = np.square(b1train)+np.square(b2train)
Xnormvalid = np.square(b1valid)+np.square(b2valid)
Xnormtest = np.square(b1test)+np.square(b2test)

In [47]:
LR = linear_model.LogisticRegression()
LR.fit(Xnorm,labelstrain)

print("Raw Score: " + str(LR.score(Xnormvalid,labelsvalid)))
pred_proba = LR.predict_proba(Xnormvalid)
print("Log Loss: " + str(metrics.log_loss(labelsvalid,pred_proba)))

Raw Score: 0.71
Log Loss: 2.1881377777


In [48]:
kernels = ['rbf', 'linear', 'sigmoid']
for k in kernels:
    SVC = svm.SVC(kernel=k)
    SVC.fit(X=Xnorm,y=labelstrain)
    print("kernel = " + str(k))
    print("Raw Score: " + str(SVC.score(Xnormvalid,labelsvalid)))

kernel = rbf
Raw Score: 0.52
kernel = linear
Raw Score: 0.74
kernel = sigmoid
Raw Score: 0.52


In [62]:
Xangles = np.arctan2(b2train,b1train)
Xanglesvalid = np.arctan2(b1valid,b2valid)
Xanglestest = np.arctan2(b1test,b2test)

In [58]:
LR = linear_model.LogisticRegression()
LR.fit(Xangles,labelstrain)

print("Raw Score: " + str(LR.score(Xanglesvalid,labelsvalid)))
pred_proba = LR.predict_proba(Xanglesvalid)
print("Log Loss: " + str(metrics.log_loss(labelsvalid,pred_proba)))

Raw Score: 0.61
Log Loss: 1.69540284942


In [63]:
Xpolar = np.concatenate((Xnorm,Xangles),axis=1)
Xpolarvalid = np.concatenate((Xnormvalid,Xanglesvalid),axis=1)
Xpolartest = np.concatenate((Xnormtest, Xanglestest),axis=1)

In [56]:
LR = linear_model.LogisticRegression()
LR.fit(Xpolar,labelstrain)

print("Raw Score: " + str(LR.score(Xpolarvalid,labelsvalid)))
pred_proba = LR.predict_proba(Xpolarvalid)
print("Log Loss: " + str(metrics.log_loss(labelsvalid,pred_proba)))

Raw Score: 0.71
Log Loss: 2.01272251272


## Now let's try a Neural Network. Start with 1 hidden layer and relu

In [93]:
#define parameters
batch_size = 64
regularization = 0.001
num_hidden = 1024
learning_rate=0.01
num_steps=1000

Xs = [X]#, Xpolar]
Xvalids = [Xvalid]#, Xpolarvalid]
Xtests = [Xtest]#, Xpolartest]

for X,Xv,Xt in zip(Xs,Xvalids,Xtests):
    num_features = X.shape[1]
    graph = tf.Graph()
    with graph.as_default():
        
        #input training data
        tf_X = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_features))
        tf_y = tf.placeholder(dtype=tf.float32, shape=(batch_size))
        
        #constants
        tf_Xvalid = tf.constant(Xv,dtype=tf.float32)
        tf_Xtest = tf.constant(Xt,dtype=tf.float32)
        
        #hidden layer (relu is the default activation)
        tf_hidden = tf.contrib.layers.fully_connected(inputs = tf_X, num_outputs = num_hidden, reuse=False, scope ='hidden')
        
        #output
        tf_output = tf.contrib.layers.fully_connected(inputs = tf_hidden, num_outputs = 1, 
                                                      activation_fn= tf.nn.sigmoid, reuse =False, scope = 'output')
        tf_vars   = tf.trainable_variables() 
        tf_loss = tf.reduce_mean(tf.losses.log_loss(labels=tf.expand_dims(tf_y,1), predictions=tf_output)
                             )+tf.add_n([ tf.nn.l2_loss(v) for v in tf_vars if 'bias' not in v.name]) *regularization
        tf_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(tf_loss)
        
        #output for validation/testing sets
        tf_valid_output = tf.contrib.layers.fully_connected(inputs =tf.contrib.layers.fully_connected(inputs = tf_Xvalid, num_outputs = num_hidden, reuse=True, scope='hidden'),
                                                           num_outputs = 1, activation_fn = tf.nn.sigmoid, reuse =True, scope= 'output')
        tf_test_output = tf.contrib.layers.fully_connected(inputs =tf.contrib.layers.fully_connected(inputs = tf_Xtest, num_outputs = num_hidden, reuse=True, scope='hidden'),
                                                           num_outputs = 1, activation_fn = tf.nn.sigmoid, reuse =True, scope= 'output')
        
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            offset = (step * batch_size) % (X.shape[0] - batch_size)
            batch_X = X[offset:(offset + batch_size), :]
            batch_labels = labelstrain[offset:(offset + batch_size)]
            feed_dict = {tf_X: batch_X, tf_y: batch_labels}
            
            _, loss = session.run([tf_optimizer, tf_loss], feed_dict=feed_dict)
            
            if(step%100 == 0):
                print(loss)
            
            


Initialized
7.23589
9.50061
10.0024
9.49686
7.48023
8.73759
10.2468
7.47466
6.46543
8.47834
